In [ ]:
import equinox as eqx
import gym
import jax
import jax.numpy as jnp
import jax.random as jrandom
import numpy as np
import optax
import sys
import timeit
import wandb

from argparse import Namespace
from functools import partial
from jax import grad, jit, vmap
from typing import Sequence

from jax_learning.buffers.ram_buffers import NextStateNumPyBuffer
from jax_learning.buffers.utils import batch_flatten, to_jnp

In [ ]:
wandb.init(project="test_jax_rl", group="cartpole-dqn_test")
wandb.define_metric("episodic_return", summary="max")

In [ ]:
cfg_dict = {
    # Environment setup
    "env": "CartPole-v1",
    "seed": 0,
    "render": False,
    
    # Experiment progress
    "load_step": 0,
    "log_interval": 5000,
    
    # Learning hyperparameters
    "max_timesteps": 1000000,
    "buffer_size": 1000000,
    "buffer_warmup": 1000,
    "num_gradient_steps": 1,
    "batch_size": 64,
    "lr": 3e-4,
    "max_grad_norm": 10.,
    "gamma": 0.99,
    "update_frequency": 4,
    "target_update_frequency": 1,
    "tau": 0.005, # This is for polyak averaging of target network
    "omega": 0.5, # This is for residual gradient: 1 for semi-gradient
    
    # Epsilon greedy hyperparameters
    "init_eps": 1.0,
    "min_eps": 0.02,
    "eps_decay": 0.9999,
    "eps_warmup": 1000,
    
    # Model architecture
    "hidden_dim": 64,
    "num_hidden": 1,
    
    # Evaluation
    "eval_cfg": {
        "max_episodes": 100,
        "seed": 1,
        "render": True,
    }
}
cfg = Namespace(**cfg_dict)
eval_cfg = Namespace(**cfg.eval_cfg)
wandb.config = cfg_dict

In [ ]:
np.random.seed(cfg.seed)

In [ ]:
env = gym.make(cfg.env)

In [ ]:
cfg.obs_dim = env.observation_space.shape
cfg.act_dim = (env.action_space.n,)
cfg.action_space = "discrete"

In [ ]:
cfg.h_state_dim = (1,)
cfg.rew_dim = (1,)

In [ ]:
cfg

In [ ]:
cfg.buffer_rng = np.random.RandomState(cfg.seed)
cfg.env_rng = np.random.RandomState(cfg.seed)
cfg.agent_key, cfg.model_key = jrandom.split(jrandom.PRNGKey(cfg.seed), num=2)
eval_cfg.env_rng = np.random.RandomState(eval_cfg.seed)

In [ ]:
def interact(env, agent, buffer, cfg):
    max_timesteps = cfg.max_timesteps
    log_interval = cfg.log_interval
    render = env.render if cfg.render else lambda: None
    env_rng = cfg.env_rng
    
    obs = env.reset(seed=env_rng.randint(0, sys.maxsize))
    h_state = agent.reset()
    ep_return = 0.
    ep_i = 0
    ep_len = 0
    tic = timeit.default_timer()
    for timestep_i in range(max_timesteps):
        timestep_dict = {
            "timestep": timestep_i,
        }
        act, next_h_state = agent.compute_action(obs, h_state, timestep_dict)
        next_obs, rew, done, info = env.step(act)
        render()
        buffer.push(obs, h_state, act, rew, done, info, next_obs, next_h_state)
        agent.learner.update(next_obs, next_h_state, timestep_dict)
        obs = next_obs
        ep_return += rew
        ep_len += 1
        
        if done:
            obs = env.reset(seed=env_rng.randint(0, sys.maxsize))
            h_state = agent.reset()
            timestep_dict["episodic_return"] = ep_return
            timestep_dict["episode"] = ep_i
            timestep_dict["episode_length"] = ep_len
            ep_return = 0.
            ep_len = 0
            ep_i += 1
            
        if (timestep_i + 1) % log_interval == 0:
            toc = timeit.default_timer()
            timestep_dict["time_diff"] = toc - tic
            tic = timeit.default_timer()
            
        wandb.log(timestep_dict)
        
def evaluate(env, agent, cfg):
    max_episodes = cfg.max_episodes
    render = env.render if cfg.render else lambda: None
    env_rng = cfg.env_rng

    for ep_i in range(max_episodes):
        tic = timeit.default_timer()
        obs = env.reset(seed=env_rng.randint(0, sys.maxsize))
        h_state = agent.reset()
        ep_return = 0.
        ep_len = 0
        done = False
        while not done:
            timestep_dict = {
                "eval_timestep": ep_len
            }
            act, next_h_state = agent.deterministic_action(obs, h_state)
            obs, rew, done, info = env.step(act)
            render()
            ep_return += rew
            ep_len += 1

            if done:
                toc = timeit.default_timer()
                timestep_dict["eval_episodic_return"] = ep_return
                timestep_dict["eval_episode"] = ep_i
                timestep_dict["eval_episode_length"] = ep_len
                timestep_dict["eval_per_episode_time"] = toc - tic

            wandb.log(timestep_dict)

In [ ]:
class ParameterizedSoftmaxQ(eqx.Module):
    obs_dim: int
    act_dim: int

    def __init__(self, obs_dim, act_dim):
        self.obs_dim = int(np.product(obs_dim))
        self.act_dim = int(np.product(act_dim))

    def greedy_action(self, obs, h_state):
        q_val, h_state = self.q_values(obs, h_state)
        acts = jnp.argmax(q_val, axis=-1)
        return acts, q_val[acts].reshape((1,)), h_state
    
    def random_action(self, obs, h_state, key):
        q_val, h_state = self.q_values(obs, h_state)
        acts = jrandom.categorical(key=key, logits=q_val, axis=-1)
        return acts, q_val[acts].reshape((1,)), h_state
    

class LinearQ(ParameterizedSoftmaxQ):
    linear: eqx.nn.Linear
    bias: jnp.ndarray

    def __init__(self, obs_dim, act_dim, key):
        super().__init__(obs_dim, act_dim)
        self.linear = eqx.nn.Linear(self.obs_dim, self.act_dim, use_bias=False, key=key)
        self.bias = jnp.zeros(self.act_dim)

    def q_values(self, obs, h_state):
        return self.linear(obs) + self.bias, h_state
    

class MLPQ(ParameterizedSoftmaxQ):
    weights: Sequence[eqx.nn.Linear]
    biases: Sequence[jnp.ndarray]

    @property
    def num_hidden(self):
        return len(self.weights) - 1

    def __init__(self, obs_dim, act_dim, hidden_dim, num_hidden, key):
        super().__init__(obs_dim, act_dim)
        self.weights = [eqx.nn.Linear(self.obs_dim, hidden_dim, use_bias=False, key=key)]
        self.biases = [jnp.zeros(hidden_dim)]
        for _ in range(num_hidden - 1):
            key, _ = jrandom.split(key, num=2)
            self.weights.append(eqx.nn.Linear(hidden_dim, hidden_dim, use_bias=False, key=key))
            self.biases.append(jnp.zeros(hidden_dim))
            
        key, _ = jrandom.split(key, num=2)
        self.weights.append(eqx.nn.Linear(hidden_dim, self.act_dim, use_bias=False, key=key))
        self.biases.append(jnp.zeros(self.act_dim))

    def q_values(self, obs, h_state):
        x = obs
        for layer_i in range(self.num_hidden):
            x = jax.nn.relu(self.weights[layer_i](x) + self.biases[layer_i])
        x = self.weights[-1](x) + self.biases[-1]
        return x, h_state

In [ ]:
class EpsilonGreedyAgent:
    def __init__(self, model, learner, init_eps, min_eps, eps_decay, eps_warmup, key):
        self._model = model
        self._learner = learner
        self._eps = init_eps
        self._init_eps = init_eps
        self._min_eps = min_eps
        self._eps_decay = eps_decay
        self._eps_warmup = eps_warmup
        self._key = key

    @property
    def model(self):
        return self._model
        
    @property
    def learner(self):
        return self._learner
        
    def deterministic_action(self, obs, h_state):
        return self.greedy_action(obs, h_state)
        
    def greedy_action(self, obs, h_state):
        action, val, next_h_state = self.model.greedy_action(obs, h_state)
        return np.asarray(action), np.asarray(next_h_state)
        
    def compute_action(self, obs, h_state, info, overwrite_rng_key=True):
        new_key, curr_key = jrandom.split(self._key)
        if jrandom.bernoulli(key=curr_key, p=self._eps):
            val, next_h_state = self.model.q_values(obs, h_state)
            action = jrandom.randint(curr_key, shape=(1,), minval=0, maxval=val.shape[-1]).item()
            info["exploration_strategy"] = 0
        else:
            action, val, next_h_state = self.model.greedy_action(obs, h_state)
            info["exploration_strategy"] = 1

        if overwrite_rng_key:
            self._key = new_key
            if self._eps_warmup > 0:
                self._eps_warmup -= 1
            else:
                self._eps = max(self._eps * self._eps_decay, self._min_eps)

        return np.asarray(action), np.asarray(next_h_state)
    
    def reset(self):
        if hasattr(self.model, "reset"):
            return self.model.reset()
        return np.array([0.], dtype=np.float32)

In [ ]:
def one_step_bellman_optimality_error(q_curr_pred, act, q_next_pred, rew, done, gamma):
    q_target = rew + (1 - done) * (gamma * jnp.max(q_next_pred))
    return q_curr_pred[act] - q_target


@eqx.filter_grad(has_aux=True)
def compute_q_learning_loss(models, obss, h_states, acts, rews, dones, next_obss, next_h_states, gammas):
    (model, target_model) = models
    q_curr_preds, _ = jax.vmap(model.q_values)(obss, h_states)
    q_next_preds, _ = jax.vmap(target_model.q_values)(next_obss, next_h_states)
    
    td_errors = jax.vmap(one_step_bellman_optimality_error)(q_curr_preds, acts, q_next_preds, rews, dones, gammas)
    loss = jnp.mean(td_errors ** 2)
    return loss, {
        "loss": loss,
        "max_q_next": jnp.max(q_next_preds),
        "min_q_next": jnp.min(q_next_preds),
        "mean_q_next": jnp.mean(q_next_preds),
        "max_q_curr": jnp.max(q_curr_preds),
        "min_q_curr": jnp.min(q_curr_preds),
        "mean_q_curr": jnp.mean(q_curr_preds),
        "max_td_error": jnp.max(td_errors),
        "min_td_error": jnp.min(td_errors),
    }

@eqx.filter_jit
def make_step(model,
              target_model,
              opt,
              opt_state,
              obss,
              h_states,
              acts,
              rews,
              dones,
              next_obss,
              next_h_states,
              gammas,
              omega):
    grads, update_info = compute_q_learning_loss((model, target_model),
                                                 obss,
                                                 h_states,
                                                 acts,
                                                 rews,
                                                 dones,
                                                 next_obss,
                                                 next_h_states,
                                                 gammas)
    
    (model_grads, target_model_grads) = grads
    grads = jax.tree_map(lambda g, tg: g * omega + tg * (1 - omega),
                         model_grads,
                         target_model_grads)
    
    updates, opt_state = opt.update(grads, opt_state)
    model = eqx.apply_updates(model, updates)
    return model, opt_state, (grads, model_grads, target_model_grads), update_info


class QLearning:
    def __init__(self, model, target_model, opt, buffer, cfg):
        self._model = model
        self._target_model = target_model
        self._opt = opt
        self._opt_state = self._opt.init(model)
        self._buffer = buffer
        self._cfg = cfg
        
        self._step = cfg.load_step
        
        self._batch_size = cfg.batch_size
        self._buffer_warmup = cfg.buffer_warmup
        self._num_gradient_steps = cfg.num_gradient_steps
        self._gamma = cfg.gamma
        self._tau = cfg.tau
        self._update_frequency = cfg.update_frequency
        self._target_update_frequency = cfg.target_update_frequency
        self._omega = cfg.omega

    @property
    def model(self):
        return self._model
    
    @property
    def target_model(self):
        return self._target_model
        
    @property
    def buffer(self):
        return self._buffer
    
    @property
    def opt(self):
        return self._opt
    
    @property
    def opt_state(self):
        return self._opt_state
        
    def update(self, next_obs, next_h_state, update_info):
        self._step += 1
        
        if self._step <= self._buffer_warmup or (self._step - 1 - self._buffer_warmup) % self._update_frequency != 0:
            return

        update_info["mean_loss"] = 0.
        update_info["mean_q_curr"] = 0.
        update_info["mean_q_next"] = 0.
        update_info["max_q_curr"] = -np.inf
        update_info["max_q_next"] = -np.inf
        update_info["min_q_curr"] = np.inf
        update_info["min_q_next"] = np.inf
        for update_i in range(self._num_gradient_steps):
            obss, h_states, acts, rews, dones, next_obss, next_h_states, _, _, _ = self.buffer.sample_with_next_obs(batch_size=self._batch_size,
                                                                                                                    next_obs=next_obs,
                                                                                                                    next_h_state=next_h_state)

            acts = acts.astype(np.int64)
            gammas = np.ones(self._batch_size) * self._gamma
            
            (obss, h_states, acts, rews, dones, next_obss, next_h_states, gammas) = to_jnp(*batch_flatten(obss,
                                                                                                          h_states,
                                                                                                          acts,
                                                                                                          rews,
                                                                                                          dones,
                                                                                                          next_obss,
                                                                                                          next_h_states,
                                                                                                          gammas))
            model, opt_state, grads, curr_update_info = make_step(model=self.model,
                                                                  target_model=self.target_model,
                                                                  opt=self.opt,
                                                                  opt_state=self.opt_state,
                                                                  obss=obss,
                                                                  h_states=h_states,
                                                                  acts=acts,
                                                                  rews=rews,
                                                                  dones=dones,
                                                                  next_obss=next_obss,
                                                                  next_h_states=next_h_states,
                                                                  gammas=gammas,
                                                                  omega=self._omega)

            self._model = model
            self._opt_state = opt_state
            
            if self._step % self._target_update_frequency == 0:
                self._target_model = jax.tree_map(lambda p, tp: p * self._tau + tp * (1 - self._tau),
                                                  self.model,
                                                  self.target_model)
            
            update_info["mean_loss"] += curr_update_info["loss"].item() / self._num_gradient_steps
            update_info["mean_q_curr"] += curr_update_info["mean_q_curr"].item() / self._num_gradient_steps
            update_info["mean_q_next"] += curr_update_info["mean_q_next"].item() / self._num_gradient_steps
            update_info["max_q_curr"] = max(update_info["max_q_curr"], curr_update_info["max_q_curr"].item())
            update_info["max_q_next"] = max(update_info["max_q_next"], curr_update_info["max_q_next"].item())
            update_info["min_q_curr"] = min(update_info["min_q_curr"], curr_update_info["min_q_curr"].item())
            update_info["min_q_next"] = min(update_info["min_q_next"], curr_update_info["min_q_next"].item())

In [ ]:
buffer = NextStateNumPyBuffer(
    buffer_size=cfg.buffer_size,
    obs_dim=cfg.obs_dim,
    h_state_dim=cfg.h_state_dim,
    act_dim=(1,) if cfg.action_space == "discrete" else cfg.act_dim,
    rew_dim=cfg.rew_dim,
    rng=cfg.buffer_rng,
)

model = MLPQ(obs_dim=cfg.obs_dim,
                act_dim=cfg.act_dim,
                hidden_dim=cfg.hidden_dim,
                num_hidden=cfg.num_hidden,
                key=cfg.model_key)

target_model = MLPQ(obs_dim=cfg.obs_dim,
                    act_dim=cfg.act_dim,
                    hidden_dim=cfg.hidden_dim,
                    num_hidden=cfg.num_hidden,
                    key=cfg.model_key)

opt = optax.chain(
    optax.clip_by_global_norm(cfg.max_grad_norm),  # Clip by the gradient by the global norm
    optax.scale_by_adam(),  # Use the updates from adam
    optax.scale(-1.0) # Gradient descent
)

learner = QLearning(model=model,
                    target_model=target_model,
                    opt=opt,
                    buffer=buffer,
                    cfg=cfg)

agent = EpsilonGreedyAgent(model=model,
                           learner=learner,
                           init_eps=cfg.init_eps,
                           min_eps=cfg.min_eps,
                           eps_decay=cfg.eps_decay,
                           eps_warmup=cfg.eps_warmup,
                           key=cfg.agent_key)

In [ ]:
%wandb

In [ ]:
interact(env, agent, buffer, cfg)

In [ ]:
evaluate(env, agent, eval_cfg)

In [ ]:
assert 0

In [ ]:
buffer

In [ ]:
buffer.observations

In [ ]:
np.roll(buffer.next_observations, 1, axis=0)

In [ ]:
np.concatenate((buffer.observations - np.roll(buffer.next_observations, 1, axis=0), buffer.dones), axis=1)

In [ ]:
buffer.sample_with_next_obs(3, buffer.next_observations[19], buffer.hidden_states[0])